In [11]:
import os
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import io

In [12]:
from googleapiclient.discovery import build

In [13]:
from googleapiclient.errors import HttpError

In [14]:
from googleapiclient.http import MediaIoBaseDownload

In [15]:
SCOPES = ["https://www.googleapis.com/auth/drive"]

In [16]:
creds = None

In [17]:
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

In [18]:
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        print("Refreshing token")
        creds.refresh(Request())
    else:
        print("generating token")
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
    
    print("Saving token")
    with open("token.json", "w") as token:
        token.write(creds.to_json())
        token.close()

In [19]:
def download_file(parent_path, file_data, service):
    try:
        this_path = os.path.join(parent_path, file_data['name'])
        if os.path.exists(this_path):
            return
        request = service.files().get_media(fileId=file_data['id'])
        fh = io.FileIO(this_path, mode='wb')
        downloader = MediaIoBaseDownload(fh, request, chunksize=32*1024*1024)

        done = False
        while done is False:
            status, done = downloader.next_chunk()
            if status:
                print("Download %d%%." % int(status.progress() * 100))
        print("Download Complete!")
    except HttpError as error:
        print(F'An error occurred: {error}')
        
def download_folder(parent_path, folder_data, depth, service):
    if depth > 5:
        print(f'Depth is greater than 5 ({depth}), returning')
        return
    this_path = os.path.join(parent_path, folder_data['name'])
    os.makedirs(this_path, exist_ok=True )
    #print(f"this_path:{this_path}")
    folder_id = folder_data['id']
    response = service.files().list(q=f"'{folder_id}' in parents",
                                   spaces='drive').execute()
    #print(f"response in download folder:{response}")
    for r in response["files"]:
        print(f"this_path:{this_path} : {r['name']}: {r['mimeType']}, {r['id']}")
        if r['mimeType'] == "application/vnd.google-apps.folder":
            download_folder(this_path, r, depth + 1, service)
        if r['mimeType'] == "application/zip":
            download_file(this_path, r, service)
               
        
try:
    service = build("drive", "v3", credentials=creds)
    response = service.files().list(q="name='oneminutedata' and mimeType='application/vnd.google-apps.folder'",
                                   spaces='drive').execute()
    print(f"response:{response}")
    folder_id = response["files"][0]['id']
    response = service.files().list(q=f"'{folder_id}' in parents",
                                   spaces='drive').execute()
    for r in response["files"]:
        print(f"top: {r['name']}: {r['mimeType']}")
        if r['name'] in map(str, range(2017,2097)):
            download_folder('data', r, 0, service)
except HttpError as ex:
    print(f"ex:{ex}")
    

response:{'kind': 'drive#fileList', 'incompleteSearch': False, 'files': [{'kind': 'drive#file', 'mimeType': 'application/vnd.google-apps.folder', 'id': '0B8e3dtbFwQWUZ1I5dklCMmE5M2M', 'name': 'oneminutedata', 'resourceKey': '0-__hqGabgp_MZvBFGo140Xg'}]}
top: 2022: application/vnd.google-apps.folder
this_path:data/2022 : DEC: application/vnd.google-apps.folder, 1VMPs3GfQmh2nWOYe1Tlvg-E3cb5q1WhH
this_path:data/2022/DEC : 09DEC.zip: application/zip, 14X-nQvfM76HjocHqenak3u6OfXZPFeja
this_path:data/2022/DEC : 08DEC.zip: application/zip, 14EDtl6SRW1gt5QnHRhIgkpzFpaRr5cMo
this_path:data/2022/DEC : 07DEC.zip: application/zip, 13c-nhZSElDH1-gb6qRMjnCBYXsYR92IY
this_path:data/2022/DEC : 06DEC.zip: application/zip, 12aL05fQNTApXFXuVRqXA8JCNyRLFwCOX
this_path:data/2022/DEC : 05DEC.zip: application/zip, 11ZxYIR5nbdfdYAfuXvUGNO461Xg-U0Vr
this_path:data/2022/DEC : 02DEC.zip: application/zip, 11E1Z_96Hx9U5xeswoHPO66SCqm275Zbx
this_path:data/2022/DEC : 01DEC.zip: application/zip, 10F79zclRkYcfPAwTa2Ap

this_path:data/2022/JUL : IntradayData_JUL2022.zip: application/zip, 11w7DDdgxx9g0w_u2r69xx7AkT8GlbcqJ
this_path:data/2022/JUL : 29JUL.zip: application/zip, 11n60gR_EiPC07TpQg7olOCuSGf46Opye
this_path:data/2022/JUL : 28JUL.zip: application/zip, 11iegLG6DYVb1qDK6yvlun5GxDk-IdLUS
this_path:data/2022/JUL : 27JUL.zip: application/zip, 11dugJdW5u7-OAL286GQuSvc8E1kUBnDW
this_path:data/2022/JUL : 26JUL.zip: application/zip, 11dHsmEmU-bdUsWuqY6W3EJnzBCtPLkSK
this_path:data/2022/JUL : 25JUL.zip: application/zip, 11b6Y_Va8q8CxODFGUHi4xaXKzhKIHb9v
this_path:data/2022/JUL : 22JUL.zip: application/zip, 11b2Un7fWqs24g3bGqOIVrugAtJz8JjDV
this_path:data/2022/JUL : 21JUL.zip: application/zip, 11ZXj7FE2uaR_zM7PM_qaUfRM4gCmH5M8
this_path:data/2022/JUL : 20JUL.zip: application/zip, 11V2uWPLLjGigchq9nfNqwXuOd32-TCKO
this_path:data/2022/JUL : 19JUL.zip: application/zip, 11R4XiUWmjQkmq42nX-Vag75h8y2G0rzV
this_path:data/2022/JUL : 18JUL.zip: application/zip, 11NVn62bphXLyHX2mZXmo508gr1PuWZ1k
this_path:data/20

this_path:data/2022/FEB : IntradayData_FEB2022.zip: application/zip, 1cjsrI_AmxJ5ybivIMlOID5gBrC21T3n4
this_path:data/2022/FEB : 28FEB.zip: application/zip, 1ch49_WFyG66-yAjtwfSfjhzQNsOiOpCO
this_path:data/2022/FEB : 25FEB.zip: application/zip, 1caTMAquYYMN3Em6P1vGj8HFFaPd0WW-l
this_path:data/2022/FEB : 24FEB.zip: application/zip, 1cK9lIv0jdYV1c8IXgeDK0urBTxiMXpY-
this_path:data/2022/FEB : 23FEB.zip: application/zip, 1cFNzTs6Eawn6Kn_vH0BJBSgVgvfhpj2W
this_path:data/2022/FEB : 22FEB.zip: application/zip, 1cF0sQY4h3092bRwyguu3YBWq99Qk2n0l
this_path:data/2022/FEB : 21FEB.zip: application/zip, 1QDbZFmzoaMjzUaV3BSjpWpcdTAsK6znI
this_path:data/2022/FEB : 18FEB.zip: application/zip, 1Pq8Sjyo6dP65_cLh4at2SjKR69rAeQwk
this_path:data/2022/FEB : 17FEB.zip: application/zip, 1Por3e04EuV_zDvk_sd1XWhDAol04gMbJ
this_path:data/2022/FEB : 16FEB.zip: application/zip, 1PTpt1dnH8iF1GvKrJMxnx6myLq8dREuE
this_path:data/2022/FEB : 15FEB.zip: application/zip, 1PQFwbBhPzbP6Cx5ha3dF4AqsTn-a12Zv
this_path:data/20

this_path:data/2021/OCT : IntradayData_OCT2021.zip: application/zip, 121aLvsduOLYYl_TIblZG55ppPYSk7Aqe
this_path:data/2021/OCT : 29OCT.zip: application/zip, 11idS_dDN0M98mHLfzDQXYXT81X7mBoW4
this_path:data/2021/OCT : 28OCT.zip: application/zip, 11hbO1tk6gqDOKskw0AYUO7j5HSDMkAQQ
this_path:data/2021/OCT : 27OCT.zip: application/zip, 11Rw5f89k4S5IEV11_w8DTOfuyk6bt96F
this_path:data/2021/OCT : 26OCT.zip: application/zip, 11PT9711-XNKPk9hT7bEXllkxd0doyBLo
this_path:data/2021/OCT : 25OCT.zip: application/zip, 11OY_SlSJ6Z4UOy_lgP30Y66-HAkU5v94
this_path:data/2021/OCT : 22OCT.zip: application/zip, 11LAt3vMHT9Zy-fICXgPOBdBvfha2ORS6
this_path:data/2021/OCT : 21OCT.zip: application/zip, 117d4WFZI6OVW0UDf4r8pqmELhfauyMut
this_path:data/2021/OCT : 20OCT.zip: application/zip, 10ue3fsJ63HpodCTowW7XMtO0jLVmKpLM
this_path:data/2021/OCT : 19OCT.zip: application/zip, 10sAllqlbBDJI06_snRpG9heRTKGCoL8v
this_path:data/2021/OCT : 18OCT.zip: application/zip, 10rrLvZyklTdqArmnNGRMGvDPk7sgRDmn
this_path:data/20

this_path:data/2021/MAY : IntradayData_MAY2021.zip: application/zip, 1Fjtpjmo_8LE_G4BYBxPSgjsF-8CQk07q
this_path:data/2021/MAY : 31MAY.zip: application/zip, 1-V_65OYbJojgoidXBTYL7TdjgeZEudhe
this_path:data/2021/MAY : 28MAY.zip: application/zip, 1xvvDRryrXCfUx3R-Q1UkGw7EIEPKaEBa
this_path:data/2021/MAY : 27MAY.zip: application/zip, 1xXwv5R-Asm2JOEp1hRmIN9R_N7X4rGzo
this_path:data/2021/MAY : 26MAY.zip: application/zip, 1uRWwPpn4xDz9EEc-x45IM2s5dtIsSxZ6
this_path:data/2021/MAY : 25MAY.zip: application/zip, 13iTI4-1JY4Ct9T5VtgemOdnfrj9kmptT
this_path:data/2021/MAY : 24MAY.zip: application/zip, 1JQS63LOvMpWunaeoKD8I9eAzP9nq_8Mt
this_path:data/2021/MAY : 21MAY.zip: application/zip, 16cp63GfIq84ctl_sWpp0Ue7EPxsFvRdR
this_path:data/2021/MAY : 20MAY.zip: application/zip, 1R7lGu5KwDU8vjhCKtHuD2KPcR-NG60Ov
this_path:data/2021/MAY : 19MAY.zip: application/zip, 13GH6t2yiYe8MQ1Evu2bucjo0T1tSWZG9
this_path:data/2021/MAY : 18MAY.zip: application/zip, 1pkc_UhmWnjkZeGbTxOoqKjYEeRP7KmUN
this_path:data/20

...

this_path:data/2019/JUN : NIFTY50_JUN2019.zip: application/zip, 1w2sinCGIZn6V-Yj6XeS8w3KFPorD89S3
this_path:data/2019/JUN : 28JUN.zip: application/zip, 1I0xHUAKy0HNHscsZ1vNSxYKu-Tm_wZcD
this_path:data/2019/JUN : 27JUN.zip: application/zip, 1CWMBoQJAiSHrrjKwWffGA_T02oHOaVZg
this_path:data/2019/JUN : 26JUN.zip: application/zip, 1gfkrNrtMDKZaifaftPlfNhRLmJg0E-_w
this_path:data/2019/JUN : 25JUN.zip: application/zip, 1efuT2-wNwcqeT_UJM5k_xgMxCZwufGfj
this_path:data/2019/JUN : 24JUN.zip: application/zip, 1CpcmUqxMOioAvS1N3AVB9ZmJnVFH9gZt
this_path:data/2019/JUN : 21JUN.zip: application/zip, 19bKaHSbGtcrOi3ED5Tj0EyGew1BVkM20
this_path:data/2019/JUN : 20JUN.zip: application/zip, 12StifWbTJMlrUQiK29AMKNXHcalHI3q3
this_path:data/2019/JUN : 19JUN.zip: application/zip, 1LZGHo3kql5cg72cM6B-uARRh3t9qkjVl
this_path:data/2019/JUN : 18JUN.zip: application/zip, 1j9lIvR32rNy1Xn2AAjdrMNxuMW6jglc6
this_path:data/2019/JUN : 17JUN.zip: application/zip, 1FSYNbOZVppudNt9gx84NAqG0b9QUTpWG
this_path:data/2019/JU

this_path:data/2019/JAN : Intraday1M_JAN2019.zip: application/zip, 10-573xuNgBSowJSstHmvaNOCizDuGSVj
this_path:data/2019/JAN : 31JAN.zip: application/zip, 1UdbM4EJ5wahOV7VP9RgjmlgYlIuDOsJ4
this_path:data/2019/JAN : 30JAN.zip: application/zip, 1Qb-Lf3gz5mugNbKxNpZal7goRGm7SR-v
this_path:data/2019/JAN : 29JAN.zip: application/zip, 11tKd2tyOOblzIdYhC7kqTIxWZbIogE51
this_path:data/2019/JAN : 28JAN.zip: application/zip, 1HqsRzGNL2c7XRRZz22NMxvfeinr1Phnj
this_path:data/2019/JAN : 25JAN.zip: application/zip, 1DYGN2-Z5mRSLiWv3A5VC39vou0zLlguf
this_path:data/2019/JAN : 24JAN.zip: application/zip, 1eXt9itR3etX3EbKGO3fHdRXMjbqvJclG
this_path:data/2019/JAN : 23JAN.zip: application/zip, 1ZJOiWMu3N1XqLVtQF3SBF92mKLub73_y
this_path:data/2019/JAN : 22JAN.zip: application/zip, 1C5Exi0uzoUZbT2CgbGBSikprASmvq87_
this_path:data/2019/JAN : 21JAN.zip: application/zip, 1wq5jdhIGwgAOZ2w0otN8nw3zBbT2gHe_
this_path:data/2019/JAN : 07JAN.zip: application/zip, 1ipTJm_jFX6vquBycPQRCQbY_iyl-XP-X
this_path:data/2019

this_path:data/2018/SEP : NIFTY50_SEP2018(with_OI).zip: application/zip, 1JCPquUBAGT_hSl8ZHvLRItUa5KWbTHd4
this_path:data/2018/SEP : 28SEP.zip: application/zip, 1HhqpvQv5ZPv0lLxmE4H0ZoEvcSXY9ZQM
this_path:data/2018/SEP : 27SEP.zip: application/zip, 1w8lQL2JK3r885WJdKya5wr94qJfh-JiA
this_path:data/2018/SEP : 26SEP.zip: application/zip, 1tXN9Z7kyuR8qN8Wrh11DpFrI7cRuQ31s
this_path:data/2018/SEP : 25SEP.zip: application/zip, 1Dhb10b5M13z7NYD5Fg2s5_FYXUh64X8C
this_path:data/2018/SEP : 24SEP.zip: application/zip, 1LYqE7KcnJxMzW7-aNaW5iCdCiDe6YHZN
this_path:data/2018/SEP : 21SEP.zip: application/zip, 1zUMKN-2BpT3TaAkcsJH6ObZhhbemPhf_
this_path:data/2018/SEP : 19SEP.zip: application/zip, 1gPfPoSoMLXj5tZUqp4N61Yez63UTlqKZ
this_path:data/2018/SEP : 18SEP.zip: application/zip, 1BoJ3gopRIKHxCZ6bKOulhbbXDfciaKTE
this_path:data/2018/SEP : 17SEP.zip: application/zip, 15qt4ExnYgs4SM2UcmkVjJOXN9cQ04zsY
this_path:data/2018/SEP : 14SEP.zip: application/zip, 1KBPkz_D7ITBBeIdFOTBg8cOn5oyMrfiO
this_path:dat

this_path:data/2018/JUN : NIFTY50_JUN2018.zip: application/zip, 1cOdga-1Rpe1CKVJen-CtPQU4FifzGnld
this_path:data/2018/JUN : 29JUN.zip: application/zip, 18rhAQg6zWu9rbSHyQwxTEB4UN3Q_xn7l
this_path:data/2018/JUN : 28JUN.zip: application/zip, 1wgbNrjkSUeOdibuaqsVxPA3bG-OA0Vib
this_path:data/2018/JUN : 27JUN.zip: application/zip, 1MoetV-GYWH0tBwpFoqfMKItgHrgpKWiy
this_path:data/2018/JUN : 26JUN.zip: application/zip, 1uF3Mq5zoW-oi9UepXsAk0_fvHPKfLdi1
this_path:data/2018/JUN : 25JUN.zip: application/zip, 1MxJ96t4qIyaN73J9OuK-C_u5rPHNJeS5
this_path:data/2018/JUN : 22JUN.zip: application/zip, 1UqJ9cu1n25fmfoW2t6e-LwmB7c4CDNyf
this_path:data/2018/JUN : 21JUN.zip: application/zip, 1jzL0y8POVdDDqjfzMbmiCYH2DdPGgEng
this_path:data/2018/JUN : 20JUN.zip: application/zip, 1d9T7096QOiVAGT4rfv2pVlGU0JMrIEEv
this_path:data/2018/JUN : 19JUN.zip: application/zip, 17wJpDtjg3yZz2NRw0ReFpRjWWUyRTEt7
this_path:data/2018/JUN : 18JUN.zip: application/zip, 1NQSt6mVKH4Wx3eL8NMGmFzxtYhoFRRsM
this_path:data/2018/JU

this_path:data/2018/FEB : NIFTY50_FEB2018.zip: application/zip, 1-2vYgGhKZcjA60K0PUFiY4nuOx4yrCW2
this_path:data/2018/FEB : 28FEB.zip: application/zip, 1f0xlAenDoIKfmBbBtUcnJxre425mSVZx
this_path:data/2018/FEB : 27FEB.zip: application/zip, 1TcrHuX6-966Lhw-3c2pHT6CIZr-n6m9u
this_path:data/2018/FEB : 26FEB.zip: application/zip, 12CDSl2NvHujt0JYWkygblFI9KmbVKADX
this_path:data/2018/FEB : 23FEB.zip: application/zip, 1g7VqCw6iJdp93EnrXZMWJ5keDpNN-knN
this_path:data/2018/FEB : 22FEB.zip: application/zip, 12WCORd3wHsiaIIAkKmaCpVHPdP1-iYxV
this_path:data/2018/FEB : 21FEB.zip: application/zip, 151e10Q4VgGAANPCHcRwT44DRIWU1K9g3
this_path:data/2018/FEB : 20FEB.zip: application/zip, 1Do84WSaB2444nky3kniGxQw3GBhmdU3D
this_path:data/2018/FEB : 19FEB.zip: application/zip, 1uC6XPIAVBr0qWUOhfRIETbLWDueBubwk
this_path:data/2018/FEB : 16FEB.zip: application/zip, 1dDM0Dew9lCoFYb7JlAiaHlwX14AZE66o
this_path:data/2018/FEB : 15FEB.zip: application/zip, 1GjFB4XCp7qpHExFn5VQ_HmcRD0eCWku4
this_path:data/2018/FE